In [6]:
from keras.applications.vgg16 import VGG16

vgg16 = VGG16(include_top=True, weights='imagenet', input_tensor=None, input_shape=None)
vgg16.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [1]:
import h5py

path = '../../.keras/models/vgg16_weights_tf_dim_ordering_tf_kernels.h5'

f = h5py.File(path, 'r')
layer_names = list(f.keys())
layer_names

['block1_conv1',
 'block1_conv2',
 'block1_pool',
 'block2_conv1',
 'block2_conv2',
 'block2_pool',
 'block3_conv1',
 'block3_conv2',
 'block3_conv3',
 'block3_pool',
 'block4_conv1',
 'block4_conv2',
 'block4_conv3',
 'block4_pool',
 'block5_conv1',
 'block5_conv2',
 'block5_conv3',
 'block5_pool',
 'fc1',
 'fc2',
 'flatten',
 'predictions']

In [2]:
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from keras.models import Model

x = h = Input((224,224,3))
h = Conv2D(64,(3,3),activation='relu',padding='same', name='c1_1')(h)
h = Conv2D(64,(3,3),activation='relu',padding='same', name='c1_2')(h)
h = MaxPooling2D()(h)
h = Conv2D(128,(3,3),activation='relu',padding='same', name='c2_1')(h)
h = Conv2D(128,(3,3),activation='relu',padding='same', name='c2_2')(h)
h = MaxPooling2D()(h)
h = Conv2D(256,(3,3),activation='relu',padding='same', name='c3_1')(h)
h = Conv2D(256,(3,3),activation='relu',padding='same', name='c3_2')(h)
h = Conv2D(256,(3,3),activation='relu',padding='same', name='c3_3')(h)
h = MaxPooling2D()(h)
h = Conv2D(512,(3,3),activation='relu',padding='same', name='c4_1')(h)
h = Conv2D(512,(3,3),activation='relu',padding='same', name='c4_2')(h)
h = Conv2D(512,(3,3),activation='relu',padding='same', name='c4_3')(h)
h = MaxPooling2D()(h)
h = Conv2D(512,(3,3),activation='relu',padding='same', name='c5_1')(h)
h = Conv2D(512,(3,3),activation='relu',padding='same', name='c5_2')(h)
h = Conv2D(512,(3,3),activation='relu',padding='same', name='c5_3')(h)
h = MaxPooling2D()(h)
h = Flatten()(h)
h = Dense(4096,activation='relu', name='fc_1')(h)
h = Dense(4096,activation='relu', name='fc_2')(h)
y = Dense(1000,activation='softmax', name='fc_3')(h)

model = Model(x,y)

Using TensorFlow backend.


In [3]:
model_layer_names = []
for layer in model.layers:
    model_layer_names.append(layer.get_config()['name'])
model_layer_names

['input_1',
 'c1_1',
 'c1_2',
 'max_pooling2d_1',
 'c2_1',
 'c2_2',
 'max_pooling2d_2',
 'c3_1',
 'c3_2',
 'c3_3',
 'max_pooling2d_3',
 'c4_1',
 'c4_2',
 'c4_3',
 'max_pooling2d_4',
 'c5_1',
 'c5_2',
 'c5_3',
 'max_pooling2d_5',
 'flatten_1',
 'fc_1',
 'fc_2',
 'fc_3']

In [4]:
if True:
    corr_layers = {
        'c1_1':'block1_conv1',
        'c1_2':'block1_conv2',
        'c2_1':'block2_conv1',
        'c2_2':'block2_conv2',
        'c3_1':'block3_conv1',
        'c3_2':'block3_conv2',
        'c3_3':'block3_conv3',
        'c4_1':'block4_conv1',
        'c4_2':'block4_conv2',
        'c4_3':'block4_conv3',
        'c5_1':'block5_conv1',
        'c5_2':'block5_conv2',
        'c5_3':'block5_conv3'
    }
else:
    corr_layers = {
        'block1_conv1':'c1_1',
        'block1_conv2':'c1_2',
        'block2_conv1':'c2_1',
        'block2_conv2':'c2_2',
        'block3_conv1':'c3_1',
        'block3_conv2':'c3_2',
        'block3_conv3':'c3_3',
        'block4_conv1':'c4_1',
        'block4_conv2':'c4_2',
        'block4_conv3':'c4_3',
        'block5_conv1':'c5_1',
        'block5_conv2':'c5_2',
        'block5_conv3':'c5_3'
    }

In [7]:
for model_layer_name in corr_layers:
    for layer in model.layers:
        if layer.get_config()['name']==model_layer_name:
            pretrained_layer_name = corr_layers[model_layer_name]
            print(list(f[pretrained_layer_name].keys()))
            w = f[pretrained_layer_name][pretrained_layer_name+'_W_1:0']
            b = f[pretrained_layer_name][pretrained_layer_name+'_b_1:0']
            layer.set_weights([w,b])

['block3_conv2_W_1:0', 'block3_conv2_b_1:0']
['block2_conv1_W_1:0', 'block2_conv1_b_1:0']
['block5_conv1_W_1:0', 'block5_conv1_b_1:0']
['block2_conv2_W_1:0', 'block2_conv2_b_1:0']
['block3_conv3_W_1:0', 'block3_conv3_b_1:0']
['block4_conv2_W_1:0', 'block4_conv2_b_1:0']
['block4_conv1_W_1:0', 'block4_conv1_b_1:0']
['block4_conv3_W_1:0', 'block4_conv3_b_1:0']
['block1_conv1_W_1:0', 'block1_conv1_b_1:0']
['block5_conv2_W_1:0', 'block5_conv2_b_1:0']
['block5_conv3_W_1:0', 'block5_conv3_b_1:0']
['block1_conv2_W_1:0', 'block1_conv2_b_1:0']
['block3_conv1_W_1:0', 'block3_conv1_b_1:0']
